In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def gurobi_solve(Q):
    n = len(Q)
    
    # Create a new model
    model = gp.Model("QUBO")

    # Create variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    
    # Create auxiliary variables and add constraints to linearize quadratic terms
    y = {}
    for i in range(n):
        for j in range(i + 1, n):
            y[i, j] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}_{j}")
            model.addConstr(y[i, j] <= x[i])
            model.addConstr(y[i, j] <= x[j])
            model.addConstr(y[i, j] >= x[i] + x[j] - 1)
    
    # Set the objective
    obj = gp.quicksum(2*Q[i, j] * y[i, j] for i in range(n) for j in range(i+1,n))
    obj += gp.quicksum(Q[i, i] * x[i] for i in range(n))
    model.setObjective(obj, GRB.MINIMIZE)

    model.Params.TimeLimit = 100  #limit the time to 100 seconds per instance (free to adjust)

    # Optimize the model
    model.optimize()

    # Retrieve the solution
    solution = [x[i].x for i in range(n)]
    best_value = model.objVal
    #return solution
    return best_value,solution

In [5]:
# Import solvers
from Gurobi import gurobi_solve
from Dwave import dwave_solve
from FNN import fnn_solve
from QCED import QCED_solve

# Load the data
import pickle
inputFile = './QUBO_Data/15_QUBO_instances.data'
fd = open(inputFile, 'rb')
data = pickle.load(fd)

# Example 15-variable random QUBO matrix 
Q = data[0][0]
min_cost = data[1][0]

# Gurobi
cost, best_sol = gurobi_solve(Q)
print(f'percentage error: {100*abs((cost-min_cost)/(min_cost))}\n solution vector: {best_sol}')

# D-Wave (API token required)
#token = "D-Wave API token"
#cost, best_sol = dwave_solve(Q,token)
#print(f'percentage error: {100*abs((cost-min_cost)/(min_cost))}\n solution vector: {best_sol}')

# FNN
cost, best_sol = fnn_solve([Q],7,[min_cost])

# QCED 
coords = np.zeros((4,2))
for i in range(4):
    coords[i,0] = i%2
    coords[i,1] = i//2
q_resource = {"q":4,"T":1000,"coords":coords,"omega_max":0.01,"delta_max":1}

result = QCED_solve(Q,q_resource)

Set parameter TimeLimit to value 100
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 4700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 315 rows, 120 columns and 735 nonzeros
Model fingerprint: 0xe445cc9f
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -4.3765020
Found heuristic solution: objective -101.9615655
Presolve removed 152 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 120 columns, 373 nonzeros
Variable types: 0 continuous, 120 integer (120 binary)

Root relaxation: objective -2.008243e+02, 103 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

In [3]:
Q

tensor([[-4.3765,  6.0103, -1.6644, -2.1272, -1.8264,  3.1592, -3.9985, -2.5698,
          5.7069,  3.3243, -2.8847, -2.7078, -5.6281,  1.0293, -3.4899],
        [ 6.0103, -4.2095,  0.3870,  4.0490, -4.4923, -4.7958, -0.2908, -5.8983,
          6.5155,  0.2436,  0.6006, -0.5616, -2.1671, -9.3391,  0.8390],
        [-1.6644,  0.3870, -2.8082, -5.2927, -9.2541,  2.2156,  3.4300,  1.4817,
          0.5320, -5.2207,  0.8692, -1.1455, -3.3206, -9.2085, -1.5092],
        [-2.1272,  4.0490, -5.2927,  1.9968, -6.4774,  0.2013, -1.4795, -2.5272,
         -0.4741, -1.3803, -4.9422, -0.4327,  5.6578, -2.4317,  2.7507],
        [-1.8264, -4.4923, -9.2541, -6.4774,  6.1777,  5.9246, -7.0541,  0.5656,
          4.4719, -0.2620, -1.8247,  2.3553, -3.6626,  4.3850,  2.6654],
        [ 3.1592, -4.7958,  2.2156,  0.2013,  5.9246,  1.1823,  0.7418, -4.6479,
          3.0333, -0.1497, -5.5820, -1.2355, -8.8413,  4.7233, -2.6350],
        [-3.9985, -0.2908,  3.4300, -1.4795, -7.0541,  0.7418, -0.4112, -5.4